In [ ]:
import numpy as np
import glob
import pickle
import seaborn as sns
import cv2
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
import tensorflow as tf
import pandas as pd
from os import listdir
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt

In [ ]:
file_path ='pv-aug/ML Project/Plant Dataset/Plant_leave_diseases_dataset_without_aug'
plant_directory = listdir(file_path)
# print(len(plant_directory))
# print(plant_directory)
plant_directory

In [ ]:
# return all file paths that match file_path+'/**/*.*'
plantfilepath = list(glob.glob(file_path  + '/**/*.*')) 
# print(plantfilepath)
print(len(plantfilepath))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], plantfilepath))
# print(labels)
print(len(labels))

# Labels and its path
filepath = pd.Series(plantfilepath, name='Filepath').astype(str)
# print(type(filepath))
# print(filepath)

labels = pd.Series(labels, name='Label')

# print(type(labels))

# each instance along with its file path
# concatenating along the columns (axis=1) to form a DataFrame
data = pd.concat([filepath, labels], axis=1)
data = data.sample(frac=1).reset_index(drop=True)
data

In [ ]:
default_image_size = tuple((256, 256))
image_size = 0
directory_root = 'pv-aug/ML Project/Plant Dataset'
width=256
height=256
depth=3

In [ ]:
# def convert_image_to_array(image_dir):
#     try:
#         image = cv2.imread(image_dir)
#         if image is not None :
#             image = cv2.resize(image, default_image_size)   
#             return img_to_array(image)
#         else :
#             return np.array([])
#     except Exception as e:
#         print(f"Error : {e}")
#         return None
    
    
datagen = ImageDataGenerator( 
        rotation_range = 40, 
#         featurewise_center=True,
#         shear_range = 0.2, 
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range = 0.2, 
        horizontal_flip = True, 
        vertical_flip= True,
        brightness_range = (0.5, 1.5)
) 

In [ ]:
# image_list keeps the array format of image
# label_list keeps the name of the species and disease
image_list, label_list = [], []
# list of names of all the files present in the specified path
try:
    print("Fetching images ...")
    root_dir = listdir(directory_root)
#     print("1", root_dir)
    for directory in root_dir :
#         print("2", directory)
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        print("plant_disease_folder_list")
#         print(plant_disease_folder_list)
        print(len(plant_disease_folder_list))

        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
            
           
            
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    img = load_img(image_directory)  
                    x = img_to_array(img) 
                    i = 0
                    
                    aug_dir = "pv-aug/ML Project/aug/" + plant_disease_folder
                    print(aug_dir)
                    if not os.path.isdir(aug_dir):
                        print('new directry has been created')
                        os.makedirs(aug_dir)

                    # Reshape the input image 
                    x = x.reshape((1, ) + x.shape) 
                    # generate 15 new augmented images 
                    for batch in datagen.flow(x, batch_size = 1, 
                                      save_to_dir = aug_dir,  
                                      save_prefix ='aug' + image,
                                      save_format ='jpg'):
                        i += 1
                        if i > 15: 
                            break
                            
                            
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("Image loading completed")  
except Exception as e:
    print(f"Error : {e}")